In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Set the display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [33]:
# URL del documento de Google Sheets en formato CSV
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQbESqM5y6bh-vL8VtNFc-2kzV3U0s0Ibqbi39sYe6vmhpL5s1nVCZgX6p1YVq53TpIBwPmhq4vLLUc/pub?gid=0&single=true&output=csv"
# Leer el CSV en un DataFrame de pandas
df = pd.read_csv(url)

In [34]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6049 entries, 0 to 6048
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   DEPARTAMENTO           6049 non-null   object
 1   PROVINCIA              6049 non-null   object
 2   DISTRITO               6049 non-null   object
 3   UBIGEO                 6049 non-null   int64 
 4   FECHA_CORTE            6049 non-null   int64 
 5   SEXO                   6049 non-null   object
 6   GRUPOS_POR_EDAD        6049 non-null   object
 7   DIAGNOSTICOS_CIE       6049 non-null   object
 8   CATEGORIA_DIAGNOSTICO  6049 non-null   object
 9   TOTAL_PACIENTES        6049 non-null   int64 
dtypes: int64(3), object(7)
memory usage: 472.7+ KB


,UBIGEO,FECHA_CORTE,TOTAL_PACIENTES
count,6049.0,6049.0,6049.000000
mean,150101.0,20230615.0,2.610845
std,0.0,0.0,6.264612
min,150101.0,20230615.0,1.000000
25%,150101.0,20230615.0,1.000000
50%,150101.0,20230615.0,1.000000
75%,150101.0,20230615.0,2.000000
max,150101.0,20230615.0,247.000000


El primer analisis exploratorio de los datos nos muestra variables las cuales de acuerdo a la bibliografia podemos definir de la siguiente forma:

**1. DEPARTAMENTO**: Esta columna contiene el nombre del departamento en Perú donde se registraron los datos. Los departamentos son divisiones administrativas y geográficas del país.

**2. PROVINCIA**: Aquí se encuentra el nombre de la provincia dentro del departamento donde se registraron los datos. Las provincias son subdivisiones de los departamentos.

**3. DISTRITO**: Este campo representa el nombre del distrito dentro de la provincia donde se registraron los datos. Los distritos son subdivisiones de las provincias.

**4. UBIGEO**: El código UBIGEO es un identificador único para ubicaciones geográficas en Perú, que generalmente abarca un distrito específico.

**5. FECHA_CORTE**: Esta columna parece ser una fecha que indica cuándo se registraron los datos. Sería útil confirmar si está en formato de fecha y, si es así, considerar su uso en análisis temporales.

**6. SEXO**: Indica el género de los pacientes registrados, generalmente dividido en categorías como "Masculino" y "Femenino".

**7. GRUPOS_POR_EDAD**: Esta columna parece contener información sobre la edad de los pacientes, posiblemente dividida en grupos de edades específicos.

**8. DIAGNOSTICOS_CIE**: Aquí se encuentran los códigos CIE (Clasificación Internacional de Enfermedades) utilizados para representar los diagnósticos médicos. Cada código corresponde a una condición médica específica.

**9. CATEGORIA_DIAGNOSTICO**: Esta columna podría agrupar los diagnósticos en categorías más amplias o clasificarlos de alguna manera. Sería importante entender cómo se define esta categorización.

**10. TOTAL_PACIENTES**: Indica el número total de pacientes registrados para una combinación específica de departamento, provincia, distrito, fecha de corte, género, grupo de edad, diagnóstico y categoría de diagnóstico.

### Técnicas de imputación para tratar con nulls

Se usara como Target a la variable CATEGORIA_DIAGNOSTICO, de acuerdo al analisis de datos, ninguna variable contiene valores nulos, por lo que no se realizara un tratamiento con nulls. Asimismo, se usaran tambien las variables DIAGNOSTICOS_CIE, SEXO y GRUPOS_POR_EDAD.

### Tratamiento de outliers

Realizacion el tratamiento de los datos categoricos. Label Encoding.

In [45]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
gender_mapping = {'MASCULINO': 0, 'FEMENINO': 1}
age_group_mapping = {
    'MENORES DE 1 AÑO': 0,
    'DE 1 A 4 AÑOS': 1,
    'DE 5 A 9 AÑOS': 2,
    'DE 10 A 14 AÑOS': 3,
    'DE 15 A 19 AÑOS': 4,
    'DE 20 A 24 AÑOS': 5,
    'DE 25 A 29 AÑOS': 6,
    'DE 30 A 34 AÑOS': 7,
    'DE 35 A 39 AÑOS': 8,
    'DE 40 A 44 AÑOS': 9,
    'DE 45 A 49 AÑOS': 10,
    'DE 50 A 54 AÑOS': 11,
    'DE 55 A 59 AÑOS': 12,
    'DE 60 A 64 AÑOS': 13,
    'DE 65 AÑOS A MÁS': 14
}
df['SEXO'] = df['SEXO'].replace(gender_mapping)
df['GRUPOS_POR_EDAD'] = df['GRUPOS_POR_EDAD'].replace(age_group_mapping)
df['SEXO'] = df['SEXO'].replace(gender_mapping)
df['GRUPOS_POR_EDAD'] = df['GRUPOS_POR_EDAD'].replace(age_group_mapping)

# Aplicando Label Encoding para CATEGORIA_DIAGNOSTICO y el target
label_encoder = LabelEncoder()
df['CATEGORIA_DIAGNOSTICO'] = label_encoder.fit_transform(df['CATEGORIA_DIAGNOSTICO'])
label_encoder1 = LabelEncoder()
df['DIAGNOSTICOS_CIE'] = label_encoder1.fit_transform(df['DIAGNOSTICOS_CIE'])

Estandarizacion de Datos

In [50]:
from sklearn.preprocessing import StandardScaler

# Escpecificando el target y los features
target_column = 'DIAGNOSTICOS_CIE'
feature_columns = ['SEXO', 'GRUPOS_POR_EDAD', 'CATEGORIA_DIAGNOSTICO']  # Add other columns you want to include as features

X = df[feature_columns]
y = df[target_column]

# Creando el modelo de estandarizaciòon
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)

# Convirtiendo la estandarizaciòn en conjunto de entrenamiento y testeo
X_standardized = pd.DataFrame(X_standardized, columns=feature_columns)
X_train, X_test, y_train, y_test = train_test_split(X_standardized, y, test_size=0.2, random_state=42)

Entrenamiento del Modelo KNN y Impresiòn de Mètricas

In [51]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

from sklearn.metrics import classification_report

y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         1
          19       0.00      0.00      0.00         2
          20       0.00      0.00      0.00         1
          21       0.00      0.00      0.00         0
          22       0.00      0.00      0.00         0
          23       0.00      0.00      0.00         1
          24       0.00      0.00      0.00         1
          27       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

Al evaluar estos datos, es evidente que la selección de características es un reflejo del enfoque gubernamental hacia la atención detallada y categorizada de los diagnósticos relacionados con la salud de los ciudadanos. Sin embargo, sería beneficioso que futuras actualizaciones de la base de datos contemplen una mayor diversificación de las características para incluir más aspectos socioeconómicos y culturales que podrían tener un impacto significativo en la salud. Así, se fomentaría un análisis más profundo y holístico, permitiendo diseñar políticas de salud más inclusivas y eficaces. A pesar de esto, la base de datos actual representa un recurso valioso para los profesionales de la salud, investigadores y tomadores de decisiones en el ámbito de la salud pública.